[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/zdemeo/apple_watch/HEAD?labpath=running_pace_calculator)
# Pace Estimator

In [1]:
# Simple calculator for estimating running pace from linear regression model
class Calculator:

    # Initialize all widgets used in pace calculator and load linear regression model
    def __init__(self, model):
        import ipywidgets as widgets
        
        # Load linear regression model
        self.model = model
        
        style = {'description_width': 'initial'}  # Allows for longer text descriptions
        
        self.elevation = widgets.BoundedIntText(
            value=500,
            min=0,
            max=2000,
            step=1,
            description='Elevation Gain (ft):',
            disabled=False,
            style=style)

        self.temp = widgets.BoundedFloatText(
            value=50.0,
            min=20.0,
            max=100.0,
            step=0.1,
            description='Max. Temperature (deg. F):',
            disabled=False,
            style=style)

        self.grade = widgets.BoundedFloatText(
            value=10.0,
            min=0.0,
            max=60.0,
            step=0.1,
            description='Max. Grade (%):',
            disabled=False,
            style=style)

        self.hr = widgets.BoundedIntText(
            value=160,
            min=110,
            max=200,
            step=1,
            description='Average Heart Rate (BPM):',
            disabled=False,
            style=style)
        
        self.sleep = widgets.BoundedFloatText(
            value=8.0,
            min=0.0,
            max=12.0,
            step=0.1,
            description='Time Asleep (hours):',
            disabled=False,
            style=style)

        self.submit = widgets.Button(
            description='Submit',
            disabled=False,
            button_style='',
            tooltip='Estimate average pace',
            icon='check')
        self.submit.on_click(self.button_callback)
        
        self.pace = widgets.Text(
            value='',
            placeholder='',
            description='Predicted Pace (minutes/mile):',
            disabled=True,
            style=style)
    
    
    # Display widgets
    def show(self):
        from IPython.display import display
        
        display(self.elevation, self.temp, self.grade, self.hr, self.sleep, self.submit, self.pace)
    
    
    # Create dataframe from kwargs. Arbitrary indexing
    @staticmethod
    def create_dataframe(**kwargs):
        import pandas as pd
        
        return pd.DataFrame(kwargs, index=["date"])
    
    
    # Predict pace based on model         
    def button_callback(self, obj):
        
        df = self.create_dataframe(hr=self.hr.value, 
                               elev=self.elevation.value, 
                               temp=self.temp.value, 
                               grade=self.grade.value,
                               sleep=self.sleep.value)
        
        prediction = lm.predict(df)[0]
        calc_pace = self.minutes_to_pace(prediction)
        
        self.pace.value = "%s:%s" % (str(calc_pace["minutes"]).zfill(2), str(calc_pace["seconds"]).zfill(2))
        #print(calc_pace)
        

    # Convert decimal minutes to pace
    @staticmethod
    def minutes_to_pace(time):
        from math import floor

        minutes = floor(time)
        seconds = floor((time - minutes) * 60)

        return {"minutes": minutes, "seconds": seconds}

In [2]:
import pickle

lm = pickle.load(open("pace_30Jan2022.pkl", "rb"))

In [3]:
pace_calculator = Calculator(lm)
pace_calculator.show()

BoundedIntText(value=500, description='Elevation Gain (ft):', max=2000, style=DescriptionStyle(description_wid…

BoundedFloatText(value=50.0, description='Max. Temperature (deg. F):', min=20.0, step=0.1, style=DescriptionSt…

BoundedFloatText(value=10.0, description='Max. Grade (%):', max=60.0, step=0.1, style=DescriptionStyle(descrip…

BoundedIntText(value=160, description='Average Heart Rate (BPM):', max=200, min=110, style=DescriptionStyle(de…

BoundedFloatText(value=8.0, description='Time Asleep (hours):', max=12.0, step=0.1, style=DescriptionStyle(des…

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Estimate average pace')

Text(value='', description='Predicted Pace (minutes/mile):', disabled=True, placeholder='', style=DescriptionS…

{'minutes': 10, 'seconds': 51}
